In [ ]:
!source ~/.bash_profile

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from tensorflow.keras.preprocessing import image
import facenet
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [ ]:
db_path = '/Users/santhoshnarayanan/Documents/Turing/GPU/MOSIP/data/testset'
celebs  = []
 
for r, d, f in os.walk(db_path): # r=root, d=directories, f = files
    for file in f:
        if ('.jpg' in file):
            exact_path = r + "/" + file
            celebs.append(exact_path)

In [ ]:
target_shape = (160, 160)
imgs = [image.load_img(img, target_size=target_shape) for img in celebs]
X = np.float32([(np.float32(img) - 127.5) / 127.5 for img in imgs])
display(X.shape)

In [ ]:
checkpoint_path = '/Users/santhoshnarayanan/Documents/Turing/GPU/MOSIP/models/'
# create a facenet model
meta_file, ckpt_file = facenet.get_model_filenames(checkpoint_path)
print('Metagraph file: %s' % meta_file)
print('Checkpoint file: %s' % ckpt_file)

In [ ]:
def deducing_metagraph_path(meta_graph_file: str):
    match = re.search(r'^(.*)\.(data-\d*-of-\d*|index|meta)$', meta_graph_file)
    if match is not None:
        deduced_meta_graph_file = match.group(1) + '.meta'
        if not os.path.isfile(deduced_meta_graph_file):
            raise Error('\n\nMetaGraph freezing mechanism was enabled. '
                        '\n{} file does not represent MetaGraph. '
                        '\n{} path to MetaGraph was deduced, but it does not exist'
                        '\n\nModel with MetaGraph consists of 3-4 files:'
                        '\n1. model_name.meta'
                        '\n2. model_name.index'
                        '\n3. model_name.data-00000-of-00001 (digit part may vary)'
                        '\n4. checkpoint (optional)'.format(meta_graph_file, deduced_meta_graph_file))
        else:
            meta_graph_file = deduced_meta_graph_file
    else:
        raise Error('\n\nMetaGraph freezing mechanism was enabled. '
                    '\n{} file does not represent MetaGraph. '
                    '\n\nModel with MetaGraph consists of 3-4 files:'
                    '\n1. model_name.meta'
                    '\n2. model_name.index'
                    '\n3. model_name.data-00000-of-00001 (digit part may vary)'
                    '\n4. checkpoint (optional)'
                    '\n\nTo load this model, simply run:'
                    '\npython3 mo_tf.py --input_meta_graph model_name.meta'
                    ''.format(meta_graph_file))
    return meta_graph_file

def read_file_to_graph_def(graph_def: [tf.GraphDef, tf.MetaGraphDef], graph_file_name: str = "",
                           is_binary: bool = True):
    """
    Reads file to protobuf
    :param graph_def: GraphDef orr MetaGraphDef object to store the network
    :param graph_file_name: path to file with graph
    :param is_binary: flag to switch between binary and test protobuf format of graph file
    :return: GraphDef or MetaGaphDef containing the network with cleared device info.
    """
    try:
        if is_binary:
            with open(graph_file_name, "rb") as f:
                graph_def.ParseFromString(f.read())
        else:
            with open(graph_file_name, "r") as f:
                text_format.Merge(f.read(), graph_def)
        nodes_to_clear_device = graph_def.node if isinstance(graph_def, tf.GraphDef) else graph_def.graph_def.node
        for node in nodes_to_clear_device:
            node.device = ""
    except Exception as e:
        raise FrameworkError(
            'TensorFlow cannot read the model file: "{}" is incorrect TensorFlow model file. '
            '\nThe file should contain one of the following TensorFlow graphs:'
            '\n1. frozen graph in text or binary format'
            '\n2. inference graph for freezing with checkpoint (--input_checkpoint) in text or binary format'
            '\n3. meta graph'
            '\n\nMake sure that --input_model_is_text is provided for a model in text format. '
            'By default, a model is interpreted in binary format. Framework error details: {}. ' +
            refer_to_faq_msg(43),
            graph_file_name,
            str(e)
        ) from e
    return graph_def

In [ ]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        meta_graph_file = deducing_metagraph_path(os.path.join(checkpoint_path, meta_file))
        input_meta_graph_def = read_file_to_graph_def(tf.MetaGraphDef(), meta_graph_file, is_binary = True)
        # Since version 2.2 TF can fail with internal error while loading graph from .meta file.
        # It happens because some operation may has an _output_shapes attribute inconsistent with the GraphDef
        # calculated value. To avoid this problem we must delete `_output_shapes` attributes from operations
        for node in input_meta_graph_def.graph_def.node:
            if '_output_shapes' in node.attr:
                del node.attr['_output_shapes']
                    
        saver = tf.train.import_meta_graph(input_meta_graph_def)
        saver.restore(tf.get_default_session(), os.path.join(checkpoint_path, ckpt_file))
        
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        # Run forward pass to calculate embeddings
        feed_dict = {images_placeholder: X[0:80], phase_train_placeholder:False}
        rep = sess.run(embeddings, feed_dict=feed_dict)
        print(rep.shape)
            
        for i in range(1, 400):
            feed_dict = { images_placeholder: X[(80*i):(80*(i+1))], phase_train_placeholder:False}
            repA = sess.run(embeddings, feed_dict=feed_dict)
            rep = np.concatenate((rep, repA), axis=0)
            print(rep.shape)

In [ ]:
df = pd.DataFrame(rep) 
df.insert(loc = 0,
          column = 'celeb',
          value = celebs)
display(df)

In [ ]:
df.to_csv('embeddings.csv')